In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
import gurobipy as grb
import datetime
import random
np.set_printoptions(edgeitems=15,linewidth=600)

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.optimization.MIP.mip_inputs import *
from src.optimization.MIP.mip_model import *
from src.optimization.MIP.mip_outputs import *

In [2]:
start = time.time()

### Model Inputs 

In [3]:
UI_params = {
    'initial_procurement_budget':1300000,
    'procurement_budget_rate':0.03,
    'initial_operations_budget':1000000,
    'maintenance_budget_rate':0.03,
    'planning_interval':[2022,2037],
    'emissions_baseline': 2031705,#metric tons
    'emissions_target_pct_of_base':1.0,
    'min_miles_replacement_threshold':150000,#miles
    'min_vehicle_age_replacement_threshold':60,#years
    'max_vehicles_per_station':1000,
    'objective_weights':{'cost':0.70,'emissions':0.30},
}

In [4]:
data = pd.read_excel('../data/17MAR_data_template.xlsx').head(500)
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')

In [5]:
inputs = MIP_Inputs(data,UI_params)

In [6]:
outputs = MIP_Outputs(data,UI_params)
print("--- %s seconds ---" % (time.time() - start))

Using license file C:\Users\elynch\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolSolutions to 500
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 533 rows, 33017 columns and 620826 nonzeros
Model fingerprint: 0x2cc2171b
Variable types: 2 continuous, 33015 integer (33015 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [2e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
Found heuristic solution: objective 3.688203e+12
Presolve removed 15 rows and 0 columns
Presolve time: 0.17s
Presolved: 518 rows, 33017 columns, 157771 nonzeros
Variable types: 2 continuous, 33015 integer (33015 binary)
Found heuristic solution: objective 61967

In [8]:
optimal_obj,optimal_solution = outputs.get_optimal_solution()

In [9]:
alt_solutions = outputs.get_alternative_solutions()
selected_alternative_solutions = outputs.select_alternative_solutions(alt_solutions,optimal_obj)

In [15]:
options = selected_alternative_solutions

In [16]:
option1 = pd.DataFrame(options[0,'schedules'],columns=['vehicle_idx','solution_idx']+inputs.years)

In [17]:
data = data.merge(option1,on='vehicle_idx',how='left')

In [18]:
data.groupby(['county'])[inputs.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
county,,,,,,,,,,,,,,,,
Baltimore County,31,32,30,33,30,33,27,33,33,28,25,24,26,15,20,18


In [19]:
data.groupby(['dept_name'])[inputs.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
dept_name,,,,,,,,,,,,,,,,
Baltimore County Department of Library,,,,,,1,,,,,,,,,,
Board of Election Supervisors,,,,,,,,,,,1,,,,,
Bureau of Corrections,,,,,1,,,1,1,2,,,2,,,1
Cigarette Restoration/Tobacco Program,,,,,,,,,,,,1,,,,
Community Planning,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Property Management - Grounds and Rec Maintenance,,,,,1,2,,,,2,1,,,,,1
Recreation and Parks (Opns./Maintnenance),,,,,,,,,,,,,,,,1
Recreation and Parks (Recreation Services),,,,,,,,,,,,,,,,


In [20]:
data.groupby(['dept_name','vehicledescription'])[inputs.years].sum().replace({0:''})

2022  \
dept_name                                  vehicledescription                     
Baltimore County Department of Library     2002 FORD VAN CARGO'                   
Board of Election Supervisors              2008 CHEVROLET UPLANDER'               
Bureau of Corrections                      2001 GMC VAN 12PSG'                    
                                           2005 FORD CROWN VICTORIA'              
                                           2006 CHEVROLET VAN,PRISONER TP'        
...                                                                         ...   
Recreation and Parks (Recreation Services) 2001 GMC VAN CARGO'                    
State's Attorney                           2006 CHEVROLET COBALT'                 
Vehicle Operations and Maintenance         2005 FORD WRECKER'                     
                                           2007 DODGE Caravan'                    
                                           2009 FORD ROLLBACK'                    

                                                                           2023  \
dept_name                                  vehicledescription                     
Baltimore County Department of Library     2002 FORD VAN CARGO'                   
Board of Election Supervisors              2008 CHEVROLET UPLANDER'               
Bureau of Corrections                      2001 GMC VAN 12PSG'                    
                                           2005 FORD CROWN VICTORIA'              
                                           2006 CHEVROLET VAN,PRISONER TP'        
...                                                                         ...   
Recreation and Parks (Recreation Services) 2001 GMC VAN CARGO'                    
State's Attorney                           2006 CHEVROLET COBALT'                 
Vehicle Operations and Maintenance         2005 FORD WRECKER'                     
                                           2007 DODGE Caravan'                    
                                           2009 FORD ROLLBACK'                    

                                                                           2024  \
dept_name                                  vehicledescription                     
Baltimore County Department of Library     2002 FORD VAN CARGO'                   
Board of Election Supervisors              2008 CHEVROLET UPLANDER'               
Bureau of Corrections                      2001 GMC VAN 12PSG'                    
                                           2005 FORD CROWN VICTORIA'              
                                           2006 CHEVROLET VAN,PRISONER TP'        
...                                                                         ...   
Recreation and Parks (Recreation Services) 2001 GMC VAN CARGO'                    
State's Attorney                           2006 CHEVROLET COBALT'                 
Vehicle Operations and Maintenance         2005 FORD WRECKER'                     
                                           2007 DODGE Caravan'                    
                                           2009 FORD ROLLBACK'                    

                                                                           2025  \
dept_name                                  vehicledescription                     
Baltimore County Department of Library     2002 FORD VAN CARGO'                   
Board of Election Supervisors              2008 CHEVROLET UPLANDER'               
Bureau of Corrections                      2001 GMC VAN 12PSG'                    
                                           2005 FORD CROWN VICTORIA'              
                                           2006 CHEVROLET VAN,PRISONER TP'        
...                                                                         ...   
Recreation and Parks (Recreation Services) 2001 GMC VAN CARGO'                    
State's Attorney                           2006 CHEVROLET COBALT'                 
Vehic

In [21]:
data.groupby(['equipmentid'])[inputs.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
equipmentid,,,,,,,,,,,,,,,,
24399.0,,,,,,,,,1,,,,,,,
26142.0,,,,,,1,,,,,,,,,,
26145.0,,,,,,,,,,,1,,,,,
26521.0,,,,,,,,,,,,,1,,,
26526.0,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30601.0,,,1,,,,,,,,,,,,,
30602.0,,,,,,,,1,,,,,,,,
30606.0,,,,,,,,1,,,,,,,,


In [22]:
print('   acquisition_costs')
pd.DataFrame(pd.DataFrame(options[0,'acquisition_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   acquisition_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,1296500,1289600,1280000,1298000,1296600,1297400,1282700,1266000,1279300,1280500,1272500,1265500,1272000,1286900,1285100,1293600


In [23]:
print('   consumables_costs')
pd.DataFrame(pd.DataFrame(options[0,'consumables_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   consumables_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,491910,455916,412953,384196,356058,336812,317503,298677,288414,275136,260717,248817,246031,238522,227718,221653


In [24]:
print('   mx_costs')
pd.DataFrame(pd.DataFrame(options[0,'mx_costs'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   mx_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,472800,477200,478200,482400,495200,492600,499600,494400,495000,489200,485800,488400,479000,472000,461600,459200


In [25]:
print('   emissions_amts')
pd.DataFrame(pd.DataFrame(options[0,'emissions_amts'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   emissions_amts


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,490655,441179,384199,339796,297157,265270,232288,204827,186017,165074,145235,119966,104343,86937,65073,52049


In [26]:
print('   EV/Hybrid Inventory')
pd.DataFrame(pd.DataFrame(options[0,'conversions'],columns=['vehicle_idx','solution_idx']+inputs.years)[inputs.years].sum()).T.astype(int)

   EV/Hybrid Inventory


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,31,63,93,126,156,189,216,249,282,309,334,357,383,398,417,432
